In [5]:
import sys
sys.path.insert(0, '/home/jianx/search-exposure/forward_ranker/')
import torch
from train import generate_sparse
from load_data import obj_reader
import network

In [ ]:
PASSAGE_EMBEDDING = obj_reader("/home/jianx/data/results/passage_embeddings.dict")
print(PASSAGE_EMBEDDING)

In [ ]:
MODEL_PATH = "/home/jianx/data/results/100_1000_1000_0.001_256_10.model"
DEVICE = torch.device("cuda:0")
EMBED_SIZE = 256

In [ ]:
NET = network.DSSM(embed_size=EMBED_SIZE)
NET.load_state_dict(torch.load(MODEL_PATH))
NET.to(DEVICE)
NET.eval()

In [ ]:
qids = []
queries = []
while len(qids) < test_batch:
    qid = random.sample(qid_list, 1)[0]
    q_seq = query_test_dict[qid]
    if q_seq != [] and qid not in qids:
        qids.append(qid)
        queries.append(generate_sparse(q_seq).to(device))
# compute cosine similarity
result_dict = {}
for i, qid in enumerate(qids):
    top_list = top_dict[qid]
    q_embed = net(queries[i]).detach()
    q_results = {}
    for j, pid in enumerate(top_list):
        p_seq = passage_dict[pid]
        if not p_seq:
            score = -1
        else:
            p_embed = net(generate_sparse(p_seq).to(device)).detach()
            score = torch.cosine_similarity(q_embed.unsqueeze(0), p_embed.unsqueeze(0)).item()
        q_results[pid] = score
    result_dict[qid] = q_results
print(sorted(result_dict[qid].items(), key=lambda x: (x[1], [-1, 1][random.randrange(2)]), reverse=True))